# Summarization of transcripts with Langchain

In this example, we intend to create a summarizer for long transcripts. The main goal is to break the original transcript into different chunks based on context - i.e. using an unsupervised approach to identify the different topics throughout the transcript (somehow similarly to Topic Modelling) - and summarize each of these chunks. in the end, the different summaries are returned to the user.

## Step 0: Configuring the environment

This step install the necessary libraries for connecting with Galileo and the models. When GenAI workspaces are available, most of the libraries will come pre-installed

In [1]:
!pip install langchain-openai
!pip install langchain-community
!pip install langchain-huggingface
!pip install langchain
!pip install promptquality #Galileo
!pip install chromadb
!pip install sentence-transformers
!pip install openai
!pip install webvtt-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 504.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 487.6 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 496.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 209.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 465.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 434.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 404.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 220.3 kB/s eta 0:00:0000:0100:03
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled

Also, we are manually configuring Hugging Face cache, so that all downloaded models are persisted in the local folder, so they can be reused even after the workspace is restarted. This also should be provided by a future feature of AI Studio 

In [1]:
import os
os.environ["HF_HOME"] = "/home/jovyan/local/hugging_face"
os.environ["HF_HUB_CACHE"] = "/home/jovyan/local/hugging_face/hub"

## Step 1: Loading the data from the transcript

At first, we need to read the data from the transcript. As our transcript is in the .vtt format, we use a library called webvtt-py to read the content. As the text is a trancript of audio/video, it is organized in small chunks of conversation, each containing a sequential id, the time of the start and end of the chunk, and the text content (often in the form speaker:content).

From this data, we expect to extract the actual content,  while keeping reference to the other metadata - for this reason, we are loading all the data into a Pandas dataset. 

In [2]:
import webvtt
import pandas as pd

data = {
    "id": [],
    "speaker": [],
    "content": [],
    "start": [],
    "end": []
}

for caption in webvtt.read('data/sample_transcript.vtt'):
    line = caption.text.split(":")
    while len(line) < 2:
        line = [''] + line
    data["id"].append(caption.identifier)
    data["speaker"].append(line[0].strip())
    data["content"].append(line[1].strip())
    data["start"].append(caption.start)
    data["end"].append(caption.end)
    
df = pd.DataFrame(data)                    
    
    

## Step 2: Semantic chunking of the transcript
At first, we need to read the data from the transcript. As our transcript is in the .vtt format, we use a library called webvtt-py to read the content. As the text is a trancript of audio/video, it is organized in small chunks of conversation, each containing a sequential id, the time of the start and end of the chunk, and the text content (often in the form speaker:content).

From this data, we expect to extract the actual content,  while keeping reference to the other metadata - for this reason, we are loading all the data into a Pandas dataset. 

In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = embedding_model.encode(df.content)


/opt/conda/envs/summarization/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-05 23:44:56.007509: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 23:44:58.093847: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/envs/summarization/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, an

In [18]:
class SemanticSplitter():
    def __init__ (self, content, embedding_model, partition_count = 10, quantile = 0.9):
        self.content = content
        self.embedding_model = embedding_model
        self.partition_count = partition_count
        self.quantile = quantile
        self.embeddings = embedding_model.encode(content)
        self.distances = [cosine(embeddings[i - 1], embeddings[i]) for i in range(1, len(embeddings))]
        self.breaks = []
        self.centroids = []
        self.load_breaks()

    def centroid_distance(self, embedding_id, centroid_id):
        return cosine(self.embeddings[embedding], self.centroid[centroid])

    def adjust_neighbors(self):
        self.breaks = []

    def load_breaks(self, method = 'number'):
        if method == 'number':
            self.breaks = np.sort(np.argpartition(self.distances, self.partition_count)[0:self.partition_count])
        elif method == 'quantiles':
            threshold = np.quantile(self.distances, self.quantile)
            self.breaks = [i for i, v in enumerate(self.distances) if v >= threshold]
        else:
            self.breaks = []

    def get_centroid(self, beginning, end):
        return embedding_model.encode('\n'.join(self.content[beginning : end]))
    
    def load_centroids(self):
        if len(self.breaks) == 0:
            self.centroids = [self.get_centroid(0, len(self.content))]
        else:
            self.centroids = []
            beginning = 0
            for break_position in self.breaks:
                self.centroids += [self.get_centroid(beginning, break_position + 1)]
                beginning = break_position + 1
            self.centroids += [self.get_centroid(beginning, len(self.content))]

    def get_chunk(self, beginning, end):
        return '\n'.join(self.content[beginning : end])
    
    def get_chunks(self):
        if len(self.breaks) == 0:
            return [self.get_chunk(0, len(self.content))]
        else:
            chunks = []
            beginning = 0
            for break_position in self.breaks:
                chunks += [self.get_chunk(beginning, break_position + 1)]
                beginning = break_position + 1
            chunks += [self.get_chunk(beginning, len(self.content))]
        return chunks
        
    

In [19]:
splitter = SemanticSplitter(df.content, embedding_model)

In [21]:
chunks = splitter.get_chunks()

## Step 2: Using a LLM model to Summarize each chunk
In our example, we are going to summarize each individual chunk separately. This solution might be advantageous in different situations:
 * When the original text is too big , or the loaded model works with a context that is too small. In this scenario, breaking information into chunks are necessary to allow the model to be applied
 * When the user wants to make sure that all the separate topics of a conversation are covered into the summarized version. An extra step could be added to allow some verification or manual configuration of the chunks to allow the user to customize the output

To achieve this goal, we load a LLM model (that can be a cloud or a local model) and use a summarization prompt

In [ ]:
prompt_template = '''
The following text is an excerpt of the transcription of a presentation:

### 
{context} 
###

Please, produce a single paragraph summarizing the given excerpt.
'''

In [30]:
import os
from langchain_openai import OpenAI

os.environ["OPENAI_API_KEY"] = "INSERT KEY"
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")


In [ ]:
### Alternate code to connect to Hugging Face models

#import yaml
#with open('config.yaml') as file:
    #config = yaml.safe_load(file)
#huggingfacehub_api_token = config["hf_key"]
#repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
#llm = HuggingFaceEndpoint(
   #huggingfacehub_api_token=huggingfacehub_api_token,
   #repo_id=repo_id,
#)


In [1]:
### Alternate code to load local models

#from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
#from langchain_community.llms import LlamaCpp

#callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

#llm = LlamaCpp(
            #model_path="/home/jovyan/datafabric/Llama7b/ggml-model-f16-Q5_K_M.gguf",
            #n_gpu_layers=64,
            #n_batch=512,
            #n_ctx=4096,
            #max_tokens=1024,
            #f16_kv=True,  
            #callback_manager=callback_manager,
            #verbose=False,
            #stop=[],
            #streaming=False,
            #temperature=0.4,
        #)
    

## Step 6: Connect to Galileo
Through the Galileo library called Prompt Quality, we connect our API generated in the Galileo console to log in. To get your ApiKey, use this link: https://console.hp.galileocloud.io/api-keys

In [32]:
import promptquality as pq

os.environ['GALILEO_API_KEY'] = "pC_FeK6dQgygTQSMEzbHSYdsa1A_nG4QeqdjkMewamw" #your api Key
galileo_url = "https://console.hp.galileocloud.io/"
pq.login(galileo_url)

👋 You have logged into 🔭 Galileo (https://console.hp.galileocloud.io/) as rafael.borges@hp.com.


Config(console_url=Url('https://console.hp.galileocloud.io/'), username=None, password=None, api_key=SecretStr('**********'), token=SecretStr('**********'), current_user='rafael.borges@hp.com', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=Url('https://api.hp.galileocloud.io/'))

In [ ]:
pq.run(project_name="simple-prompt-with-chatgpt",
       template=my_prompt_template,
       settings=pq.Settings(model_alias='ChatGPT (16K context)',
                            temperature=0.1,
                            max_tokens=400))


Through callbacks, we choose the metrics we want to monitor via the Galileo console. We pass a list of queries to run our created chain and log in to Galileo.


In [25]:
dir(pq.Scorers)

['__class__',
 '__doc__',
 '__members__',
 '__module__',
 'chunk_attribution_utilization_luna',
 'chunk_attribution_utilization_plus',
 'completeness_luna',
 'completeness_plus',
 'context_adherence_luna',
 'context_adherence_plus',
 'context_relevance',
 'correctness',
 'pii',
 'prompt_injection',
 'prompt_perplexity',
 'sexist',
 'tone',
 'toxicity']